# Load Data & Prcessing Data


In [2]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the dataset
data_path ='/content/ad_click_dataset.csv'
data = pd.read_csv(data_path)

Mengonversi variabel kategorikal menjadi format numerik menggunakan one-hot encoding. drop_first=True untuk menghindari masalah multikolinearitas.

# Memisahkan Fitur dan Target


In [3]:
# Display the first few rows and data types
print(data.head())
print(data.dtypes)

     id full_name   age      gender device_type ad_position browsing_history  \
0   670   User670  22.0         NaN     Desktop         Top         Shopping   
1  3044  User3044   NaN        Male     Desktop         Top              NaN   
2  5912  User5912  41.0  Non-Binary         NaN        Side        Education   
3  5418  User5418  34.0        Male         NaN         NaN    Entertainment   
4  9452  User9452  39.0  Non-Binary         NaN         NaN     Social Media   

  time_of_day  click  
0   Afternoon      1  
1         NaN      1  
2       Night      1  
3     Evening      1  
4     Morning      0  
id                    int64
full_name            object
age                 float64
gender               object
device_type          object
ad_position          object
browsing_history     object
time_of_day          object
click                 int64
dtype: object


In [4]:
# Convert categorical variables if needed
data = pd.get_dummies(data, drop_first=True)

In [5]:
# Check for missing values
print(data.isnull().sum())
data.fillna(data.mean(), inplace=True)  # or dropna() if preferable


id                                  0
age                              4766
click                               0
full_name_User100                   0
full_name_User1000                  0
                                 ... 
browsing_history_Shopping           0
browsing_history_Social Media       0
time_of_day_Evening                 0
time_of_day_Morning                 0
time_of_day_Night                   0
Length: 4015, dtype: int64


In [6]:
# Define features and target
X = data.iloc[:, :-1].values  # Features
y = data.iloc[:, -1].values   # Target


Memisahkan data menjadi fitur (X) dan target (y), di mana X adalah semua kolom kecuali kolom terakhir, dan y adalah kolom terakhir.


# Membagi data (Splitting) menjadi data testing

In [7]:
# Split into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardisasi Fitur

In [8]:
# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Menstandarkan fitur agar memiliki rata-rata 0 dan deviasi standar 1

# Mode Sequential

In [9]:
# Build the Sequential model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Use 'softmax' for multi-class classification
])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Membangun model dengan beberapa layer:
 1. Input Layer: 64 neuron, fungsi aktivasi ReLU.
 2. Hidden Layers: 32 dan 16 neuron, fungsi aktivasi ReLU.
 3. Output Layer: 1 neuron dengan fungsi aktivasi sigmoid untuk klasifikasi biner.

# Mengompilasi Model

In [10]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Melatih model selama 50 epoch dengan ukuran batch 32. 20% dari data pelatihan digunakan untuk validasi.

# Melatih Model

In [11]:
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

Epoch 1/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.8034 - loss: 0.4521 - val_accuracy: 0.8456 - val_loss: 0.2904
Epoch 2/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9050 - loss: 0.1837 - val_accuracy: 0.8794 - val_loss: 0.2652
Epoch 3/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9625 - loss: 0.0937 - val_accuracy: 0.8737 - val_loss: 0.2791
Epoch 4/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9642 - loss: 0.0815 - val_accuracy: 0.8694 - val_loss: 0.2919
Epoch 5/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9630 - loss: 0.0791 - val_accuracy: 0.8569 - val_loss: 0.3145
Epoch 6/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9641 - loss: 0.0765 - val_accuracy: 0.8494 - val_loss: 0.3351
Epoch 7/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9643 - loss: 0.0789 - val_accuracy: 0.8219 - val_loss: 0.3843
Epoch 8/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9682 - loss: 0.0742 - val_accuracy: 0

Melatih model selama 50 epoch dengan ukuran batch 32. 20% dari data pelatihan digunakan untuk validasi.

# Menevaluasi Model

In [12]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=1)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7389 - loss: 0.4225


In [15]:
test_acc_percentage = test_acc * 100
print(f'Test accuracy: {test_acc_percentage:.2f}%')

Test accuracy: 74.15%


Mengevaluasi model menggunakan data pengujian dan menyimpan nilai kehilangan (loss) dan akurasi.

Mengonversi akurasi pengujian ke dalam persentase dan mencetaknya

# Mencetak Riwayat Akurasi Pelatihan dan Validasi

In [27]:
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

In [21]:
train_accuracy_percentage = [acc * 100 for acc in train_accuracy]
val_accuracy_percentage = [acc * 100 for acc in val_accuracy]

In [23]:
print("\nTraining Accuracy (in %):")
for epoch, acc in enumerate(train_accuracy_percentage):
    print(f"Epoch {epoch + 1}: {acc:.2f}%")


Training Accuracy (in %):
Epoch 1: 81.05%
Epoch 2: 92.89%
Epoch 3: 96.28%
Epoch 4: 96.45%
Epoch 5: 96.42%
Epoch 6: 96.47%
Epoch 7: 96.50%
Epoch 8: 96.52%
Epoch 9: 96.50%
Epoch 10: 96.52%
Epoch 11: 96.56%
Epoch 12: 96.56%
Epoch 13: 96.45%
Epoch 14: 96.56%
Epoch 15: 96.50%
Epoch 16: 96.53%
Epoch 17: 96.55%
Epoch 18: 96.50%
Epoch 19: 96.53%
Epoch 20: 96.48%
Epoch 21: 96.58%
Epoch 22: 96.56%
Epoch 23: 96.63%
Epoch 24: 96.47%
Epoch 25: 96.53%
Epoch 26: 96.59%
Epoch 27: 96.64%
Epoch 28: 96.64%
Epoch 29: 96.56%
Epoch 30: 96.66%
Epoch 31: 96.58%
Epoch 32: 96.61%
Epoch 33: 96.61%
Epoch 34: 96.72%
Epoch 35: 96.58%
Epoch 36: 96.61%
Epoch 37: 96.67%
Epoch 38: 96.63%
Epoch 39: 96.75%
Epoch 40: 96.61%
Epoch 41: 96.70%
Epoch 42: 96.70%
Epoch 43: 96.75%
Epoch 44: 96.75%
Epoch 45: 96.81%
Epoch 46: 96.80%
Epoch 47: 96.89%
Epoch 48: 96.81%
Epoch 49: 96.83%
Epoch 50: 96.78%


In [24]:
print("\nValidation Accuracy (in %):")
for epoch, acc in enumerate(val_accuracy_percentage):
    print(f"Epoch {epoch + 1}: {acc:.2f}%")


Validation Accuracy (in %):
Epoch 1: 84.56%
Epoch 2: 87.94%
Epoch 3: 87.37%
Epoch 4: 86.94%
Epoch 5: 85.69%
Epoch 6: 84.94%
Epoch 7: 82.19%
Epoch 8: 83.13%
Epoch 9: 83.63%
Epoch 10: 82.94%
Epoch 11: 82.63%
Epoch 12: 82.56%
Epoch 13: 80.81%
Epoch 14: 81.06%
Epoch 15: 81.75%
Epoch 16: 80.75%
Epoch 17: 78.69%
Epoch 18: 80.62%
Epoch 19: 80.50%
Epoch 20: 80.44%
Epoch 21: 79.06%
Epoch 22: 79.44%
Epoch 23: 78.69%
Epoch 24: 78.37%
Epoch 25: 77.75%
Epoch 26: 78.44%
Epoch 27: 76.56%
Epoch 28: 78.87%
Epoch 29: 78.31%
Epoch 30: 75.88%
Epoch 31: 78.94%
Epoch 32: 78.00%
Epoch 33: 75.56%
Epoch 34: 78.12%
Epoch 35: 76.56%
Epoch 36: 74.87%
Epoch 37: 76.75%
Epoch 38: 73.12%
Epoch 39: 77.44%
Epoch 40: 74.69%
Epoch 41: 74.19%
Epoch 42: 72.87%
Epoch 43: 71.94%
Epoch 44: 72.50%
Epoch 45: 73.31%
Epoch 46: 72.87%
Epoch 47: 72.81%
Epoch 48: 72.62%
Epoch 49: 71.94%
Epoch 50: 72.81%


Mencetak akurasi pelatihan dan validasi untuk setiap epoch dalam format persentase.

# Menampilkan Rata-rata Akurasi

In [25]:
avg_train_acc = sum(train_accuracy_percentage) / len(train_accuracy_percentage)
avg_val_acc = sum(val_accuracy_percentage) / len(val_accuracy_percentage)

In [26]:
print(f"\nAverage Training Accuracy: {avg_train_acc:.2f}%")
print(f"Average Validation Accuracy: {avg_val_acc:.2f}%")


Average Training Accuracy: 96.22%
Average Validation Accuracy: 78.65%


Menghitung dan mencetak rata-rata akurasi pelatihan dan validasi.